# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
user_log.createOrReplaceTempView("user_log")
spark.sql('SELECT DISTINCT(page) FROM user_log WHERE userId =""').show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [5]:
user_log.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


Failed login


# Question 3

How many female users do we have in the data set?

In [6]:
spark.sql("SELECT COUNT(userId) FROM user_log WHERE gender='F'").show()

+-------------+
|count(userId)|
+-------------+
|         3820|
+-------------+



# Question 4

How many songs were played from the most played artist?

In [7]:
spark.sql('SELECT artist, COUNT(artist) as count FROM user_log GROUP BY artist ORDER BY count DESC').show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
|Red Hot Chili Pep...|   24|
+--------------------+-----+
only showing top 20 rows



In [8]:
spark.sql('SELECT song, COUNT(song) as count FROM user_log WHERE artist = "Coldplay" GROUP BY song ORDER BY count DESC').show()

+--------------------+-----+
|                song|count|
+--------------------+-----+
|              Yellow|   21|
|              Clocks|   12|
|       The Scientist|   11|
|             Fix You|    8|
|              Shiver|    8|
|         In My Place|    4|
|             Trouble|    2|
| Life In Technicolor|    1|
|            Daylight|    1|
|       White Shadows|    1|
|                 Yes|    1|
|Now My Feet Won't...|    1|
|               Lost!|    1|
|          I Ran Away|    1|
|    Til Kingdom Come|    1|
|A Rush Of Blood T...|    1|
|     Bigger Stronger|    1|
|        Warning Sign|    1|
|Everything's Not ...|    1|
|      Glass Of Water|    1|
+--------------------+-----+
only showing top 20 rows



In [9]:
user_log.take(10)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [41]:
song_counter = udf(lambda x: 1 if x =='Home' else 0)
user_log = user_log.withColumn("songCounter", song_counter(user_log.page))

In [11]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- songCounter: string (nullable = true)



In [42]:
user_log.createOrReplaceTempView("user_log")
spark.sql('SELECT userId, page, songCounter FROM user_log WHERE userId = 1046').show()

+------+--------+-----------+
|userId|    page|songCounter|
+------+--------+-----------+
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|    Home|          1|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|  Logout|          0|
|  1046|    Home|          1|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
|  1046|NextSong|          0|
+------+--------+-----------+
only showing top 20 rows



In [43]:
from pyspark.sql import Window
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])
user_log_valid = user_log_valid.withColumn("songSum", Fsum("songCounter").over(windowval))

In [47]:
user_log_valid.createOrReplaceTempView("user_log_valid")

In [59]:
spark.sql('SELECT userId, songSum FROM user_log_valid WHERE page="NextSong" GROUP BY userId, songSum').show()

+------+-------+
|userId|songSum|
+------+-------+
|  1436|    0.0|
|  2088|    1.0|
|  2162|    0.0|
|  2162|    2.0|
|  2294|    0.0|
|  2294|    1.0|
|  2294|    2.0|
|  2294|    3.0|
|  2294|    4.0|
|  2294|    5.0|
|  2904|    0.0|
|   691|    1.0|
|  1394|    0.0|
|  1394|    1.0|
|  2275|    0.0|
|  2756|    0.0|
|  2756|    2.0|
|   451|    0.0|
|   451|    1.0|
|   800|    1.0|
+------+-------+
only showing top 20 rows



In [40]:
spark.sql('SELECT AVG(count) FROM(SELECT userId, page, COUNT(page) as count FROM user_log_valid WHERE page = "NextSong" GROUP BY userId, page)').show()

+----------------+
|      avg(count)|
+----------------+
|9.87810650887574|
+----------------+

